In [2]:
import numpy as np 
import pennylane as qml

### Codercise F.3.1 - The 3-qubit QFT

Implement the QFT for three qubits.

In [ ]:
num_wires = 3
dev = qml.device("default.qubit", wires = num_wires)

@qml.qnode(dev)
def three_qubit_QFT(basis_id):
    """A circuit that computes the QFT on three qubits.
    
    Args:
        basis_id (int): An integer value identifying the basis state to construct.
        
    Returns:
        array[complex]: The state of the qubits after the QFT operation.
    """
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width = num_wires)]
    qml.BasisStatePreparation(bits, wires=[0, 1, 2])
    
    qml.Hadamard(wires = 0)
    qml.ctrl(qml.S(wires = 0), control = 1)
    qml.ctrl(qml.T(wires = 0), control = 2)

    qml.Hadamard(wires = 1)
    qml.ctrl(qml.S(wires = 1), control = 2)

    qml.Hadamard(wires = 2)

    qml.SWAP(wires = [0, 2])
    
    return qml.state()

### Codercise F.3.2 - Swapping qubits

Implement a circuit that reverses the order of $n$ qubits using SWAP gates. You need only $n/2$ gates for this.

In [ ]:
dev = qml.device('default.qubit', wires=4)

            
def swap_bits(n_qubits):
    """A circuit that reverses the order of qubits, i.e.,
    performs a SWAP such that [q1, q2, ..., qn] -> [qn, ... q2, q1].
    
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
    """
    for i in range(int(n_qubits / 2)):
        qml.SWAP(wires=[i, n_qubits - 1 - i])

@qml.qnode(dev) 
def qft_node(basis_id, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width = n_qubits)]
    qml.BasisStatePreparation(bits, wires = range(n_qubits))
    # qft_rotations(n_qubits)
    swap_bits(n_qubits)
    return qml.state()

### Codercise F.3.3 - Putting the QFT together

Implement the circuit that performs the Hadamards and controlled rotations on $n$ qubits using qml.ControlledPhaseShift. The swap_bits operation defined in the exercise above is available to use. The result is the $n$-qubit QFT!

In [ ]:
dev = qml.device('default.qubit', wires=4)

def qft_rotations(n_qubits):
    """A circuit performs the QFT rotations on the specified qubits.
    
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
    """
    n = 0
    for i in range(n_qubits):
        qml.Hadamard(wires = n)
        for k in range(1, n_qubits - n):
            qml.ControlledPhaseShift(np.pi / 2 ** k, wires = [n, n + k])
        n += 1

@qml.qnode(dev) 
def qft_node(basis_id, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width = n_qubits)]
    qml.BasisStatePreparation(bits, wires = range(n_qubits))
    qft_rotations(n_qubits)
    swap_bits(n_qubits)
    return qml.state()

### Codercise F.3.4 - A recursive solution

Implement the circuit that performs the Hadamard operations and controlled rotations on  qubits recursively. The swap_bits operation defined earlier is available for you.

In [ ]:
dev = qml.device('default.qubit', wires = 4)

def qft_recursive_rotations(n_qubits, wire = 0):
    """A circuit that performs the QFT rotations on the specified qubits
        recursively.
        
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
        wire (int): An integer identifying the wire 
                    (or the qubit) to apply rotations on.
    """
    qml.Hadamard(wires = wire)
    for k in range(1, n_qubits):
        qml.ControlledPhaseShift(np.pi / 2 ** k, wires = [wire, wire + k])
        
    n_qubits -= 1
    wire += 1
    
    if n_qubits == 1:
        return qml.Hadamard(wires = wire)
    else:
        return qft_recursive_rotations(n_qubits, wire)

@qml.qnode(dev) 
def qft_node(basis_id, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width = n_qubits)]
    qml.BasisStatePreparation(bits, wires = range(n_qubits))
    qft_recursive_rotations(n_qubits)
    swap_bits(n_qubits)
    return qml.state()

### Codercise F.3.5 - The QFT in PennyLane

Implement the QFT using qml.QFT.

In [ ]:
dev = qml.device('default.qubit', wires = 4)

@qml.qnode(dev)
def pennylane_qft(basis_id, n_qubits):
    """A that circuit performs the QFT using PennyLane's QFT template.
    
    Args:
        basis_id (int): An integer value identifying 
            the basis state to construct.
        n_qubits (int): An integer identifying the 
            number of qubits.
            
    Returns:
        array[complex]: The state after applying the QFT to the qubits.
    """
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width = n_qubits)]
    qml.BasisStatePreparation(bits, wires = range(n_qubits))

    qml.QFT(wires = [wire for wire in range(n_qubits)])

    return qml.state()